# Bayesian Networks

author: Jacob Schreiber <br>
contact: jmschreiber91@gmail.com

Bayesian networks are a powerful inference tool, in which a set of variables are represented as nodes, and the lack of an edge represents a conditional independence statement between the two variables, and an edge represents a dependence between the two variables. One of the powerful components of a Bayesian network is the ability to infer the values of certain variables, given observed values for another set of variables. These are referred to as the 'hidden' and 'observed' variables respectively, and need not be set at the time the network is created. The same network can have a different set of variables be hidden or observed between two data points. The more values which are observed, the closer the inferred values will be to the truth.

While Bayesian networks can have extremely complex emission probabilities, usually Gaussian or conditional Gaussian distributions, pomegranate currently supports only discrete Bayesian networks. Bayesian networks are explicitly turned into Factor Graphs when inference is done, wherein the Bayesian network is turned into a biparte graph with all variables having marginal nodes on one side, and joint tables on the other.

If you didn't understand that, it's okay! Lets get down to a simple example, the Monty Hall example. The Monty Hall problem arose from the gameshow Let's Make a Deal, where a guest had to choose which one of three doors had a prize behind it. The twist was that after the guest chose, the host, originally Monty Hall, would then open one of the doors the guest did not pick and ask if the guest wanted to switch which door they had picked. Initial inspection may lead you to believe that if there are only two doors left, there is a 50-50 chance of you picking the right one, and so there is no advantage one way or the other. However, it has been proven both through simulations and analytically that there is in fact a 66% chance of getting the prize if the guest switches their door, regardless of the door they initially went with.

We can reproduce this result using Bayesian networks with three nodes, one for the guest, one for the prize, and one for the door Monty chooses to open. The door the guest initially chooses and the door the prize is behind are completely random processes across the three doors, but the door which Monty opens is dependent on both the door the guest chooses (it cannot be the door the guest chooses), and the door the prize is behind (it cannot be the door with the prize behind it).

In [1]:
from pomegranate import *

# The guests initial door selection is completely random
guest = DiscreteDistribution( { 'A': 1./3, 'B': 1./3, 'C': 1./3 } )

# The door the prize is behind is also completely random
prize = DiscreteDistribution( { 'A': 1./3, 'B': 1./3, 'C': 1./3 } )

    # Monty is dependent on both the guest and the prize. 
monty = ConditionalProbabilityTable(
        [[ 'A', 'A', 'A', 0.0 ],
         [ 'A', 'A', 'B', 0.5 ],
         [ 'A', 'A', 'C', 0.5 ],
         [ 'A', 'B', 'A', 0.0 ],
         [ 'A', 'B', 'B', 0.0 ],
         [ 'A', 'B', 'C', 1.0 ],
         [ 'A', 'C', 'A', 0.0 ],
         [ 'A', 'C', 'B', 1.0 ],
         [ 'A', 'C', 'C', 0.0 ],
         [ 'B', 'A', 'A', 0.0 ],
         [ 'B', 'A', 'B', 0.0 ],
         [ 'B', 'A', 'C', 1.0 ],
         [ 'B', 'B', 'A', 0.5 ],
         [ 'B', 'B', 'B', 0.0 ],
         [ 'B', 'B', 'C', 0.5 ],
         [ 'B', 'C', 'A', 1.0 ],
         [ 'B', 'C', 'B', 0.0 ],
         [ 'B', 'C', 'C', 0.0 ],
         [ 'C', 'A', 'A', 0.0 ],
         [ 'C', 'A', 'B', 1.0 ],
         [ 'C', 'A', 'C', 0.0 ],
         [ 'C', 'B', 'A', 1.0 ],
         [ 'C', 'B', 'B', 0.0 ],
         [ 'C', 'B', 'C', 0.0 ],
         [ 'C', 'C', 'A', 0.5 ],
         [ 'C', 'C', 'B', 0.5 ],
         [ 'C', 'C', 'C', 0.0 ]], [guest, prize] )  

# State objects hold both the distribution, and a high level name.
s1 = State( guest, name="guest" )
s2 = State( prize, name="prize" )
s3 = State( monty, name="monty" )

# Create the Bayesian network object with a useful name
network = BayesianNetwork( "Monty Hall Problem" )

# Add the three states to the network 
network.add_states( [ s1, s2, s3 ] )

# Add transitions which represent conditional dependencies, where the second node is conditionally dependent on the first node (Monty is dependent on both guest and prize)
network.add_transition( s1, s3 )
network.add_transition( s2, s3 )
network.bake()

pomegranate uses loopy belief propogation on the factor graph to calculate marginals, meaning that it is an inexact algorithm, but converges to the exact solution on Bayesian networks which have a tree structure. We can use the `predict_proba` method in order to ask questions for single data points. As a baseline, lets see what happens if we don't put in any information. This should give us the marginal of the graph, which is that everything is equally likely.

In [2]:
print network.predict_proba({})

[ {
    "frozen" : false,
    "class" : "Distribution",
    "parameters" : [
        {
            "A" : 0.33333333333333337,
            "C" : 0.33333333333333337,
            "B" : 0.33333333333333337
        }
    ],
    "name" : "DiscreteDistribution"
}
 {
    "frozen" : false,
    "class" : "Distribution",
    "parameters" : [
        {
            "A" : 0.33333333333333337,
            "C" : 0.33333333333333337,
            "B" : 0.33333333333333337
        }
    ],
    "name" : "DiscreteDistribution"
}
 {
    "frozen" : false,
    "class" : "Distribution",
    "parameters" : [
        {
            "A" : 0.33333333333333337,
            "C" : 0.33333333333333337,
            "B" : 0.33333333333333337
        }
    ],
    "name" : "DiscreteDistribution"
}]


We are returned three `DiscreteDistribution` objects, each representing the marginal distribution for each variable, in the same order they were put into the model. In this case, they represent the guest, prize, and monty variables respectively. We see that everything is equally likely. If we want to access these distributions, we can do the following:

In [3]:
marginals = network.predict_proba({})
print marginals[0].parameters[0]

{'A': 0.33333333333333337, 'C': 0.33333333333333337, 'B': 0.33333333333333337}


The first element of `marginals` is a DiscreteDistribution, with all the same operations as a normal DiscreteDistribution objects. This means that parameters[0] will return the underlying dictionary used by the distribution, which we return here.

Now lets do something different, and say that the guest has chosen door 'A'. We do this by passing a dictionary to `predict_proba` with key pairs consisting of the name of the state (in the state object), and the value which that variable has taken.

In [4]:
network.predict_proba({'guest': 'A'})

array([ {
    "frozen" : false,
    "class" : "Distribution",
    "parameters" : [
        {
            "A" : 1.0,
            "C" : 0.0,
            "B" : 0.0
        }
    ],
    "name" : "DiscreteDistribution"
},
       {
    "frozen" : false,
    "class" : "Distribution",
    "parameters" : [
        {
            "A" : 0.33333333333333337,
            "C" : 0.33333333333333337,
            "B" : 0.33333333333333337
        }
    ],
    "name" : "DiscreteDistribution"
},
       {
    "frozen" : false,
    "class" : "Distribution",
    "parameters" : [
        {
            "A" : 0.0,
            "C" : 0.5,
            "B" : 0.5
        }
    ],
    "name" : "DiscreteDistribution"
}], dtype=object)

We can see that now Monty will not open door 'A', because the guest has chosen it. At the same time, the distribution over the prize has not changed, it is still equally likely that the prize is behind each door.

Now, lets say that Monty opens door 'C' and see what happens.

In [5]:
network.predict_proba({'guest': 'A', 'monty': 'C'})

array([ {
    "frozen" : false,
    "class" : "Distribution",
    "parameters" : [
        {
            "A" : 1.0,
            "C" : 0.0,
            "B" : 0.0
        }
    ],
    "name" : "DiscreteDistribution"
},
       {
    "frozen" : false,
    "class" : "Distribution",
    "parameters" : [
        {
            "A" : 0.3333333333333334,
            "C" : 0.0,
            "B" : 0.6666666666666666
        }
    ],
    "name" : "DiscreteDistribution"
},
       {
    "frozen" : false,
    "class" : "Distribution",
    "parameters" : [
        {
            "A" : 0.0,
            "C" : 1.0,
            "B" : 0.0
        }
    ],
    "name" : "DiscreteDistribution"
}], dtype=object)

Suddenly, we see that the distribution over prizes has changed. It is now twice as likely that the car is behind the door labelled 'B'. This illustrates the somewhat famous Monty Hall problem.

Bayesian networks also have an `impute` method, where you can pass in many points at the same time, with `None` in the variables which you don't know, and it will fill them in with the maximum likelihood estimate of the value of the varaible after loopy belief propogation is run. If all values are equally likely, it will randomly choose a value, which may be sub-optimal. For an example:

In [6]:
network.impute( [['B', 'A', None],
                 ['C', 'A', None],
                 ['B', 'C', None],
                 ['A', 'B', None]])

[['B', 'A', 'C'], ['C', 'A', 'B'], ['B', 'C', 'A'], ['A', 'B', 'C']]

Networks can be trained by passing in a matrix with observation position corresponding to the state in the model, such as the following:

In [7]:
network.fit( [['A', 'B', 'C'],
              ['A', 'C', 'B'],
              ['A', 'A', 'C'],
              ['B', 'B', 'C'], 
              ['B', 'C', 'A']] )

In [8]:
print network.predict_proba({})

[ {
    "frozen" : false,
    "class" : "Distribution",
    "parameters" : [
        {
            "A" : 0.6,
            "C" : 0.0,
            "B" : 0.4
        }
    ],
    "name" : "DiscreteDistribution"
}
 {
    "frozen" : false,
    "class" : "Distribution",
    "parameters" : [
        {
            "A" : 0.20000000000000007,
            "C" : 0.39999999999999997,
            "B" : 0.39999999999999997
        }
    ],
    "name" : "DiscreteDistribution"
}
 {
    "frozen" : false,
    "class" : "Distribution",
    "parameters" : [
        {
            "A" : 0.18666666666666665,
            "C" : 0.5466666666666666,
            "B" : 0.26666666666666666
        }
    ],
    "name" : "DiscreteDistribution"
}]
